# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 12:49:17,423 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

In [3]:
df = se.read.csv("/data/clickstream.csv", header=True, sep='\t')
df.show(5)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



## Spark DF solution

In [4]:
# Define window for each session ordered by timestamp
window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

# Identify the timestamp of the first error in each session
df_with_error_timestamp = df.withColumn(
    "error_timestamp",
    F.min(F.when(F.col("event_type").like("%error%"), F.col("timestamp"))).over(window_spec)
)

# Filter out events occurring after the first error in each session
clickstream_filtered = df_with_error_timestamp.filter(
    (F.col("error_timestamp").isNull()) | (F.col("timestamp") < F.col("error_timestamp"))
)

# Add row numbers within each session 
sorted_df = clickstream_filtered.withColumn(
    "row_num",
    F.row_number().over(window_spec)
).orderBy("user_id", "session_id", "timestamp", "row_num")

# Remove consecutive duplicate event pages within the same session
deduped_df = sorted_df.withColumn(
    "prev_event_page",
    F.lag("event_page").over(window_spec)
).filter(
    (F.col("prev_event_page").isNull()) | (F.col("event_page") != F.col("prev_event_page"))
)


# Define a UDF to remove consecutive duplicates in collected list
def remove_consecutive_duplicates(event_pages):
    if not event_pages:
        return ""
    deduped_pages = [event_pages[0]]
    for page in event_pages[1:]:
        if page != deduped_pages[-1]:
            deduped_pages.append(page)
    return '-'.join(deduped_pages)


remove_duplicates_udf = F.udf(remove_consecutive_duplicates, StringType())

# Group by session, remove duplicates and form routes
routes_df = deduped_df.groupBy("user_id", "session_id").agg(
    remove_duplicates_udf(F.collect_list("event_page")).alias("route")
)

# Count occurrences of each route and sort in descending order
route_counts = routes_df.groupBy("route").count().orderBy(F.desc("count"))

top_30_routes = route_counts.limit(30)

top_30_routes.show(top_30_routes.count(), False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8184 |
|main-archive         |1113 |
|main-rabota          |1047 |
|main-internet        |897  |
|main-bonus           |870  |
|main-news            |769  |
|main-tariffs         |676  |
|main-online          |587  |
|main-vklad           |518  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |139  |
|main-bonus-rabota    |135  |
|main-news-rabota     |135  |
|main-archive-internet|132  |
|main-rabota-news     |130  |
|main-internet-rabota |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-tariffs-internet|114  |
|main-internet-bonus  |114  |
|main-news-archive    |113  |
|main-news-internet   |109  |
|main-archive-tariffs |104  |
|main-internet-news   |103  |
|main-tariffs-archive |103  |
|main-rabota-main     |94   |
+---------

## Spark SQL solution

In [5]:
# Register the DataFrame as a temporary view for SQL querying
df.createOrReplaceTempView("clickstream")

sql_query = """
WITH deduped_events AS (
    SELECT
        user_id,
        session_id,
        timestamp,
        event_page,
        event_type,
        MIN(CASE WHEN event_type LIKE '%error%' THEN timestamp END) OVER (PARTITION BY user_id, session_id) AS error_timestamp,
        LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS prev_event_page
    FROM clickstream
),
filtered_events AS (
    SELECT
        user_id,
        session_id,
        event_page
    FROM deduped_events
    WHERE (error_timestamp IS NULL OR timestamp < error_timestamp)
      AND (prev_event_page IS NULL OR event_page != prev_event_page)
),
routes AS (
    SELECT
        user_id,
        session_id,
        CONCAT_WS('-', COLLECT_LIST(event_page)) AS route
    FROM filtered_events
    GROUP BY user_id, session_id
),
route_counts AS (
    SELECT
        route,
        COUNT(*) AS count
    FROM routes
    GROUP BY route
)
SELECT route, count
FROM route_counts
ORDER BY count DESC
LIMIT 30
"""

top_30_routes = se.sql(sql_query)

top_30_routes.show(top_30_routes.count(), False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8184 |
|main-archive         |1112 |
|main-rabota          |1047 |
|main-internet        |897  |
|main-bonus           |870  |
|main-news            |769  |
|main-tariffs         |677  |
|main-online          |587  |
|main-vklad           |517  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |139  |
|main-bonus-rabota    |135  |
|main-news-rabota     |135  |
|main-archive-internet|131  |
|main-internet-rabota |129  |
|main-rabota-news     |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-internet-bonus  |115  |
|main-tariffs-internet|114  |
|main-news-archive    |113  |
|main-news-internet   |109  |
|main-archive-tariffs |104  |
|main-internet-news   |103  |
|main-tariffs-archive |103  |
|main-rabota-main     |94   |
+---------

## Spark RDD solution

In [7]:
from operator import add

rdd = sc.textFile("/data/clickstream.csv")
header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split('\t'))

data_rdd = data_rdd.map(lambda row: (row[0], row[1], row[2], row[3], row[4]))

# Filter rows with errors and mark events after error in each session
def filter_errors(session_data):
    clean_session = []
    error_occurred = False
    for row in session_data:
        if 'error' in row[2]:  # Check if event_type contains 'error'
            error_occurred = True
        if not error_occurred:
            clean_session.append(row)
    return clean_session

# Group by user and session, then filter out events after the first error
filtered_rdd = data_rdd.map(lambda x: ((x[0], x[1]), x)) \
                       .groupByKey() \
                       .flatMapValues(filter_errors) \
                       .map(lambda x: x[1])

# Sort events by timestamp within each session
sorted_rdd = filtered_rdd.map(lambda x: ((x[0], x[1]), (x[3], x[4]))) \
                         .groupByKey() \
                         .mapValues(lambda pages: sorted(enumerate(pages), key=lambda x: (x[1][1], x[0]))) \
                         .mapValues(lambda pages: [page[1][0] for page in pages])  # Extract event_page from sorted tuples


# Remove consecutive duplicate pages within each session
def remove_consecutive_duplicates(event_pages):
    deduped_pages = [event_pages[0]]
    for page in event_pages[1:]:
        if page != deduped_pages[-1]:
            deduped_pages.append(page)
    return deduped_pages


routes_rdd = sorted_rdd.mapValues(remove_consecutive_duplicates) \
                       .mapValues(lambda pages: '-'.join(pages)) \
                       .map(lambda x: x[1])

# Count occurrences of each route and sort in descending order
route_counts = routes_rdd.map(lambda route: (route, 1)) \
                         .reduceByKey(add) \
                         .sortBy(lambda x: x[1], ascending=False)

top_30_routes = route_counts.take(30)

for route, count in top_30_routes:
    print(f"{route}: {count}")

main: 8185
main-archive: 1113
main-rabota: 1047
main-internet: 897
main-bonus: 870
main-news: 769
main-tariffs: 677
main-online: 587
main-vklad: 518
main-rabota-archive: 170
main-archive-rabota: 167
main-bonus-archive: 143
main-rabota-bonus: 139
main-bonus-rabota: 135
main-news-rabota: 135
main-archive-internet: 132
main-rabota-news: 130
main-internet-rabota: 129
main-archive-news: 126
main-rabota-internet: 124
main-internet-archive: 123
main-archive-bonus: 117
main-internet-bonus: 115
main-news-archive: 113
main-tariffs-internet: 113
main-news-internet: 109
main-archive-tariffs: 104
main-internet-news: 103
main-tariffs-archive: 103
main-rabota-main: 94


In [10]:
import json

routes_dict = {route: count for route, count in top_30_routes}

with open("result_top30.json", "w") as json_file:
    json.dump(routes_dict, json_file)